In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd
from datamatch import (
    ColumnsIndex, StringSimilarity, JaroWinklerSimilarity, DateSimilarity, ThresholdMatcher
)

from lib.columns import rearrange_complaint_columns
from lib.explore import print_df
from clean.baton_rouge_pd_cprr import clean_18, clean_10_20, clean_21
from clean.baton_rouge_csd_pprr import clean_17 as clean_csd_17, clean_19 as clean_csd_19
from clean.baton_rouge_pd_pprr import clean as clean_pprr

In [2]:
df18 = clean_18()
df18

standardize_from_lookup_table: unmatched sequences:
  {'', ' (60 day rule)', 'command of temper (', '; ', 'awol ', ': ', ', ', ' & '}


,investigation_status,action,disposition,department_code,department_desc,last_name,first_name,middle_initial,middle_name,receive_year,...,receive_day,occur_year,occur_month,occur_day,tracking_number,charges,agency,data_production_year,uid,complaint_uid
0,administrative review,exonerated,exonerated,P1254,cib street crimes unit,barcelona,jesse,r,,2018,...,2,2017,12,30,2018-002,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2020,29d926d8dcd79668e3de9d5445d135f0,861636c724625ebdd796c6c29f79a1f1
1,administrative review,not sustained,not sustained,P10280,cib narcotics,acree,jason,k,,2018,...,9,2018,1,9,2018-004,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2020,a02b3a98a05c9b8fedf038032d0b094e,d8eadacd00f2799a60cc044d251c972f
2,office investigation,resigned,office investigation,P10476,op service comm center,rimes,melissa,j,,2018,...,9,2017,12,3,2018-003,2:4 abuse of sick leave - 66,Baton Rouge PD,2020,0091f0e72f48d071b19d010a764443c2,8b11f1af4416e503a359ef867a3dfe82
3,administrative review,not sustained,not sustained,P10578,cib cib,hamadeh,yuseff,g,,2018,...,10,2018,1,10,2018-005,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2020,34f7a308cfef56486d03e5fca0c15bd8,526990f4672fc74ee9612a0432cb6fef
4,administrative review,not sustained,not sustained,P10252,patrol 2nd district,pollar jr,charles,g,,2018,...,11,2018,1,11,2018-006,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2020,16d2084871ee1d18c4c91f3c65b41e21,e967764fb0f5b8fbecf515f42bf674a7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,,2019,...,30,2018,3,27,2019-003,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2020,745a0d63456a6a7bb7a1ae5e95620046,3d59fd53994f143979be30836d475c2c
137,administrative review,letter of reprimand,not sustained,P1245,patrol 2nd district,clark,kenneth,m,,2019,...,30,2018,3,27,2019-003,3:18 insubordination - 43,Baton Rouge PD,2020,745a0d63456a6a7bb7a1ae5e95620046,bce6db892cf8522c2d4f292f17f682eb
138,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,,2019,...,30,2018,3,27,2019-003,2:12 conduct unbecoming an officer - 21,Baton Rouge PD,2020,745a0d63456a6a7bb7a1ae5e95620046,576243fd83ff2b159a98adfcc6ab8526
139,administrative review,not sustained,not sustained,P10639,patrol 1st district,robertson,jason,r,,2018,...,26,2018,1,26,2018-010,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2020,6c09e90720823c54685fa9611beb3824,7cd0c62140a5c07b261d21bbea4c6778


In [3]:
df18.investigation_status.unique()

array(['administrative review', 'office investigation', 'referral'],
      dtype=object)

In [4]:
df21 = clean_21()
df21

standardize_from_lookup_table: unmatched sequences:
  set()


,received,action,disposition,complainant,investigation_status,tracking_number,receive_year,receive_month,receive_day,occur_year,...,rank_desc,first_name,last_name,middle_name,middle_initial,division_desc,agency,data_production_year,uid,complaint_uid
0,8/10/2010,letter of caution,,brpd,pa,2010-HS,2010,8,10,2010,...,NaN,dominic,distefano,,,NaN,Baton Rouge PD,2021,aa6d6e701c607f952b5c66dc01ba8682,71c3eed9f20fe92e131141f3761324a8
1,8/17/2010,letter of caution,,brpd,pa,2010-HS,2010,8,17,2010,...,NaN,glenn,hutto,,,NaN,Baton Rouge PD,2021,092300a2355af709bc6709f434927da0,ce7acbf12eb5e07d77e3b8dcffc8afe6
2,8/24/2010,letter of caution,,brpd,pa,2010-HS,2010,8,24,2010,...,NaN,steven,woodring,,,NaN,Baton Rouge PD,2021,61da07409d5adc39fb4a77fb1ac8acde,eeb925f8c8e5edbd885151d31220a4fc
3,8/24/2010,letter of caution,,brpd,pa,2010-HS,2010,8,24,2010,...,NaN,anthanor,bradford,,,NaN,Baton Rouge PD,2021,0911ff6ce87c40bacfdc6664221c76ee,92867f858ba7496362583f614b9f22bc
4,8/24/2010,letter of caution,,brpd,pa,2010-HS,2010,8,24,2010,...,NaN,paul,brown,,,NaN,Baton Rouge PD,2021,4f750d23b70d84b170cdbc36ffc36cab,f2287c39098639d590045305d95abbd4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1856,1/19/2021,,,brpd,administrative review,2021-7,2021,1,19,2021,...,NaN,jeremy,bonnet,,j,NaN,Baton Rouge PD,2021,2c77c5b4e868d08d96f252ecdb3273a0,1996e6348ff0bf67edf66cd6c4b24a7d
1857,1/19/2021,,,brpd,administrative review,2021-7,2021,1,19,2021,...,NaN,jeremy,bonnet,,j,NaN,Baton Rouge PD,2021,2c77c5b4e868d08d96f252ecdb3273a0,6fef8ae593930776b3e2d2309bee9864
1858,2/9/2021,,,brpd,administrative review,2021-9,2021,2,9,2021,...,NaN,jason,lee,,a,NaN,Baton Rouge PD,2021,9f7e0c70c49aafa51d04ebee556e171d,20e54eff579c167dc9499e0882c50010
1859,2/9/2021,,,brpd,administrative review,2021-9,2021,2,9,2021,...,NaN,jason,lee,,a,NaN,Baton Rouge PD,2021,9f7e0c70c49aafa51d04ebee556e171d,000b2921fb88a610fc6c908c65ac9aed


In [13]:
df21.receive_year.unique()

array(['2010', '2011', '2012', '2013', '', '2014', '2017', '2015', '2016',
       '2019', '2020', '2021'], dtype=object)

In [6]:
df = clean_10_20()
df

,receive_date,occur_date,officer_name,complaint,action,disposition,complainant,investigation_status,tracking_number
0,8/10/2010,5/4/2010,Dominic Distefano,Damaging Dept. Equipment,Sustained/Letter of Caution,Admin. Review,BRPD,pa,2010-HS
1,8/17/2010,1/29/2010,Glenn Hutto,Damaging Dept. Equipment,Sustained/Letter of Caution,Admin. Review,BRPD,pa,2010-HS
2,8/24/2010,3/19/2010,Steven Woodring,Damaging Dept. Equipment,Sustained/Letter of Caution,Admin. Review,BRPD,pa,2010-HS
3,8/24/2010,4/2/2010,Anthanor Bradford,Damaging Dept. Equipment,Sustained/Letter of Caution,Admin. Review,BRPD,pa,2010-HS
4,8/24/2010,3/15/2010,Paul Brown,Damaging Dept. Equipment,Sustained/Letter of Caution,Admin. Review,BRPD,pa,2010-HS
...,...,...,...,...,...,...,...,...,...
1859,1/19/2021,1/18/2021,"BONNET, JEREMY J P10542 Patrol 3rd District",2:7 Damaging Dept. Equipment - 18,NaN,-,BRPD,administrative review,2021-7
1860,1/19/2021,1/18/2021,"BONNET, JEREMY J P10542 Patrol 3rd District",2:11 Conduct Unbecoming an Officer - 21,NaN,-,BRPD,administrative review,2021-7
1861,2/9/2021,1/28/2021,"LEE, JASON A P10517 Patrol 3rd District",2:11 Conduct Unbecoming an Officer - 21,NaN,-,BRPD,administrative review,2021-9
1862,2/9/2021,1/28/2021,"LEE, JASON A P10517 Patrol 3rd District",2:13 DMVR Violation - 68,NaN,-,BRPD,administrative review,2021-9


In [9]:
df[(df.receive_date.isna() | ~df.receive_date.str.match(r'^\d{1,2}/\d{1,2}/\d{4}$'))]

,receive_date,occur_date,officer_name,complaint,action,disposition,complainant,investigation_status,tracking_number
683,-,6/4/2013,Luke Cowart,Use of Force,Exonerated,/Admin. Review,BRPD,ret,2013-39
684,-,6/4/2013,Lorenzo Coleman,Use of Force,Exonerated,"/Admin, Review",BRPD,ret,2013-39
1230,-,-,CHARLES EDDLEMAN,Abuse of sick leave,LOC,SUSTAINED,BRPD,,2015-HS


In [11]:
df.loc[(df.occur_date.isna() | ~df.occur_date.str.match(r'^\d{1,2}/\d{1,2}/\d{4}$')), 'occur_date'].unique()

array(['0/10/2010 ', '- ', '2014', 'Oct-13', '2015', 'Feb-16', '2016',
       'Jul-16', 'Jul-17', '1092019', '2019-06 ', '. '], dtype=object)